# Import libraries

In [411]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pandas.tseries.offsets import DateOffset
from pandas.tseries.frequencies import to_offset

# Get data

In [412]:
URL_DATA_CPI = 'https://storage.googleapis.com/dosm-public-economy/cpi_core.parquet'
URL_DATA_GDP = 'https://storage.googleapis.com/dosm-public-economy/gdp.parquet'
URL_DATA_UNEM = 'https://storage.googleapis.com/dosm-public-economy/labour-principalstats-monthly.parquet'
URL_DATA_FOREX = 'https://storage.googleapis.com/dosm-public-economy/exchangerates-main.parquet'
URL_DATA_PPI = 'https://storage.googleapis.com/dosm-public-economy/ppi.parquet'
CSV_MONEY = 'money.csv'

cpi = pd.read_parquet(URL_DATA_CPI)
if 'date' in cpi.columns:
    cpi['date'] = pd.to_datetime(cpi['date'])
cpi.set_index('date', inplace=True)


gdp = pd.read_parquet(URL_DATA_GDP)
if 'date' in gdp.columns:
    gdp['date'] = pd.to_datetime(gdp['date'])
gdp.set_index('date', inplace=True)
gdp = gdp.loc[gdp['series_type']=='real']
gdp.drop(columns='series_type', inplace=True)


unemployment = pd.read_parquet(URL_DATA_UNEM)
if 'date' in unemployment.columns:
    unemployment['date'] = pd.to_datetime(unemployment['date'])
unemployment.set_index('date', inplace=True)
unemployment = unemployment[['u_rate', 'p_rate', 'ep_ratio']]

forex = pd.read_parquet(URL_DATA_FOREX)
if 'date' in forex.columns:
    forex['date'] = pd.to_datetime(forex['date'])
forex.set_index('date', inplace=True)
forex = forex['myr_usd']
forex = forex.resample('MS').mean()

ppi = pd.read_parquet(URL_DATA_PPI)
if 'date' in ppi.columns:
    ppi['date'] = pd.to_datetime(ppi['date'])
ppi.set_index('date', inplace=True)

money = pd.read_csv(CSV_MONEY)
if 'date' in money.columns:
    money['date'] = pd.to_datetime(money['date'])
money.set_index('date', inplace=True)
money = money.applymap(lambda x: float(x.replace(',', '')))

In [413]:
cpi

,overall,food_beverage,clothing_footwear,housing_utilities,furnishings,health,transport,communication,recreation_culture,education,hospitality,misc
date,,,,,,,,,,,,
2015-01-01,109.3,114.8,98.6,111.0,106.8,111.7,104.8,97.2,105.3,111.1,118.6,106.2
2015-02-01,109.7,115.1,98.2,112.2,106.6,112.1,104.9,97.2,105.7,111.7,118.8,106.7
2015-03-01,109.8,115.2,98.6,112.2,106.7,112.5,105.0,97.2,105.5,111.8,119.2,107.0
2015-04-01,111.2,116.8,99.4,112.4,109.0,114.6,105.9,100.2,106.5,112.4,121.4,109.7
2015-05-01,111.7,117.2,99.5,113.3,109.4,114.9,106.3,100.3,106.7,112.3,121.9,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,126.5,143.5,93.4,129.8,124.0,125.9,118.7,97.5,116.5,122.6,141.8,119.3
2022-10-01,126.6,144.4,93.4,129.8,124.2,126.0,119.3,97.5,116.4,122.7,142.2,119.6
2022-11-01,127.1,145.4,93.4,130.1,124.3,126.4,119.1,97.5,116.6,122.9,142.9,120.0


In [414]:
gdp

,overall,supply_agri,supply_mining,supply_manufacturing,supply_construction,supply_services,supply_import_duties,demand_c,demand_g,demand_i,demand_x,demand_m,demand_nx,demand_inventory
date,,,,,,,,,,,,,,
2015-01-01,282254,21691,26879,62333,13728,154469,3154,154769,32731,74978,196992,174240,22752,-2976
2015-04-01,288975,24001,25729,65695,12927,156945,3678,154020,34967,77560,195711,175303,20408,2020
2015-07-01,297965,27685,24260,65851,14332,162177,3660,164632,35910,76447,209491,186442,23049,-2073
2015-10-01,307746,24161,26191,68500,14395,170292,4207,161678,50413,75438,215176,192793,22383,-2166
2016-01-01,294329,21208,26586,65146,14825,162558,4006,162611,33638,75036,198934,178409,20525,2519
2016-04-01,301014,22617,26313,68160,14083,165993,3848,163474,36916,82265,199813,180168,19645,-1286
2016-07-01,311510,26404,24927,68630,15478,172206,3865,174771,36718,77891,209054,183775,25279,-3149
2016-10-01,322458,23748,27542,71962,15121,179805,4280,171404,48367,76999,220354,196878,23476,2212
2017-01-01,310745,22608,26845,69022,15832,172090,4348,173330,36245,82183,218683,201682,17001,1986


In [415]:
unemployment

,u_rate,p_rate,ep_ratio
date,,,
2010-01-01,3.53,64.10,61.83
2010-02-01,3.54,62.53,60.32
2010-03-01,3.48,63.80,61.58
2010-04-01,3.03,64.47,62.52
2010-05-01,3.27,62.93,60.87
...,...,...,...
2022-08-01,3.70,69.70,67.10
2022-09-01,3.60,69.70,67.20
2022-10-01,3.60,69.70,67.20


In [416]:
forex

date
2003-12-01    0.263416
2004-01-01    0.263079
2004-02-01    0.263297
2004-03-01    0.263247
2004-04-01    0.262779
                ...   
2022-10-01    0.213468
2022-11-01    0.216540
2022-12-01    0.226724
2023-01-01    0.231233
2023-02-01    0.230356
Freq: MS, Name: myr_usd, Length: 231, dtype: float64

In [417]:
ppi

,overall,agriculture,mining,manufacturing,electricity,water
date,,,,,,
2010-01-01,98.4,92.7,99.4,98.8,99.8,99.2
2010-02-01,98.4,93.3,97.8,99.0,99.4,99.1
2010-03-01,99.9,96.1,108.6,99.2,100.5,99.2
2010-04-01,99.8,95.3,107.7,99.3,99.7,99.9
2010-05-01,98.6,94.1,91.4,99.8,100.3,101.5
...,...,...,...,...,...,...
2022-08-01,120.9,126.3,103.6,122.7,117.8,115.3
2022-09-01,119.2,119.6,98.5,121.8,117.6,116.6
2022-10-01,119.3,117.2,103.4,121.5,118.1,117.4


In [418]:
money

,M1,M2,M3
date,,,
2013-01-01,297582.2,1355381.2,1375858.0
2013-02-01,294329.3,1358466.8,1379389.6
2013-03-01,293719.6,1373101.6,1392122.9
2013-04-01,292406.8,1382444.1,1399798.9
2013-05-01,295628.3,1398166.3,1414632.1
...,...,...,...
2022-08-01,602482.5,2232643.4,2241310.6
2022-09-01,596710.8,2247122.0,2254815.7
2022-10-01,597499.6,2248503.5,2257146.3


# Data visualisation

## Consumer Price Index (CPI)

In [419]:
traces = []

for col_name in cpi.columns:
    trace = go.Scatter(x=cpi.index, y=cpi[col_name], mode='lines', name=col_name)
    traces.append(trace)

layout = go.Layout(
    title='Consumer Price Index (CPI)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='CPI')
)

fig = go.Figure(data=traces, layout=layout)

fig.show()

### Key notes:

1. *food_beverage* has been highest and has the most significant increasing trend
2. *communication* and *clothing_footwear* has been lowest and has a constant trend, however *clothing_footwear* has a slight decrease while is the opposite for *communication*
3. *overall* trend is increasing indicationg increasing of CPI

## Growth Domestic Product (GDP)

In [420]:
traces = []

for col_name in gdp.columns:
    trace = go.Scatter(x=gdp.index, y=gdp[col_name], mode='lines', name=col_name)
    traces.append(trace)

layout = go.Layout(
    title='Growth Domestic Product (gdp)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='gdp')
)

fig = go.Figure(data=traces, layout=layout)

fig.show()

### Key notes:

1. *overall* has a steady increase with a drop during COVID-19
2. *demand_x*, *demand_m*, *demand_c* and *supply services* has the strongest GDP growth with steady increament

## Employability

In [421]:
traces = []

for col_name in unemployment.columns:
    trace = go.Scatter(x=unemployment.index, y=unemployment[col_name], mode='lines', name=col_name)
    traces.append(trace)

layout = go.Layout(
    title='Employability',
    xaxis=dict(title='Date'),
    yaxis=dict(title='unemployment')
)

fig = go.Figure(data=traces, layout=layout)

fig.show()

### Key notes:

1. *u_rate* has a significant increase during COVID-19 and has been gradually decreasing. It has stabilise but is the rate is still higher than before COVID-19
2. *p_rate* and *ep_ratio* shows the same pattern and is in the rise. There was a slide drop during COVID-19

## FOREX

In [422]:
# create a line chart
fig = go.Figure(data=go.Scatter(x=forex.index, y=forex, mode='lines'))

# add chart title and axis labels
fig.update_layout(title='MYR/USD Exchange Rate', xaxis_title='Date', yaxis_title='Exchange Rate')

# show the chart
fig.show()

### Key notes:

The exchange rate had a significant drop coming into 2016 and has fluctutated up until now. It was the lowest in _Nov 2022_ perhaps due to the election of Anwar Ibrahim as the Prime Minister and coming into political stability

## Producer Price Index (PPI)

In [423]:
traces = []

for col_name in ppi.columns:
    trace = go.Scatter(x=ppi.index, y=ppi[col_name], mode='lines', name=col_name)
    traces.append(trace)

layout = go.Layout(
    title='Producer Price Index (ppi)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='ppi')
)

fig = go.Figure(data=traces, layout=layout)

fig.show()

### Key notes:

1. *agriculture* showed a spike increase after COVID-19 and has stabilised
2. *mining* is below average post COVID-19
3. all other PPI's are aligned with the *overall* PPI

## Money Supply

In [424]:
traces = []

for col_name in money.columns:
    trace = go.Scatter(x=money.index, y=money[col_name], mode='lines', name=col_name)
    traces.append(trace)

layout = go.Layout(
    title='Money Supply',
    xaxis=dict(title='Date'),
    yaxis=dict(title='money')
)

fig = go.Figure(data=traces, layout=layout)
fig.update_layout(title='Money Supply', xaxis_title='Date', yaxis_title='Millions')

fig.show()

### Key notes:

1. *M2 and M3* are similar so could consider to only take either one

# Data consolidation

In [430]:
df = pd.DataFrame()
df['CPI'] = cpi['overall']
df['GDP'] = gdp['overall']
df['GDP'].fillna(method='ffill', inplace=True)
df['u_rate'] = unemployment['u_rate']
df['p_rate'] = unemployment['p_rate']
df.dropna(inplace=True)
df['myr_usd'] = forex.loc[df.index[0]:df.index[-1]]
df['PPI'] = ppi['overall']
df['M2'] = money['M2']

In [431]:
df

,CPI,GDP,u_rate,p_rate,myr_usd,PPI,M2
date,,,,,,,
2015-01-01,109.3,282254.0,3.12,67.59,0.279938,104.1,1545765.7
2015-02-01,109.7,282254.0,3.17,68.00,0.278064,104.6,1552127.2
2015-03-01,109.8,282254.0,3.02,68.10,0.271717,103.8,1583077.5
2015-04-01,111.2,288975.0,2.97,67.89,0.275814,103.1,1572159.3
2015-05-01,111.7,288975.0,3.08,67.76,0.277996,101.6,1567577.2
...,...,...,...,...,...,...,...
2022-08-01,126.1,383775.0,3.70,69.70,0.224107,120.9,2232643.4
2022-09-01,126.5,383775.0,3.60,69.70,0.220802,119.2,2247122.0
2022-10-01,126.6,397191.0,3.60,69.70,0.213468,119.3,2248503.5


# Final data visualisation